In [1]:
# 1. Start the Spark session 
from pyspark.sql import SparkSession
import pandas as pd

spark = (
    SparkSession.builder
    .appName("EmployeeHours")
    .config("spark.driver.memory", "2g")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

In [2]:
# 2. Read the clean Parquet and handle unsupported timestamp columns
pdf = pd.read_parquet('../data/timesheet_clean.parquet')

spark_df = spark.createDataFrame(pdf)
#df = spark.read.parquet("../data/timesheet_clean.parquet").drop("Date")

# 3. Group-by & Sum
emp_hours = (
    spark_df.groupby("employee_name")
        .sum("hours_worked")
        .withColumnRenamed("sum(hours_worked)", "total_hours")
)

In [3]:
from pyspark import StorageLevel

# 4. Persist the DataFrame to avoid recomputation with a specific storage level
emp_hours.persist(StorageLevel.MEMORY_AND_DISK)

# 5. Repartition the DataFrame to optimize for writing
emp_hours = emp_hours.repartition(4)

# 6. Show the top-10 with truncation to avoid overwhelming the driver
try:
	emp_hours.show(10, truncate=True)
except Exception as e:
	print(f"Error displaying DataFrame: {e}")

# 7. Write out with optimized configurations
try:
	emp_hours.write.mode("overwrite").parquet("../data/employee_hours_spark.parquet")
except Exception as e:
	print(f"Error writing DataFrame: {e}")

# 8. Stop Spark
#spark.stop()

Error displaying DataFrame: An error occurred while calling o61.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 0.0 failed 1 times, most recent failure: Lost task 5.0 in stage 0.0 (TID 5) (DESKTOP-RTVU5EU.attlocal.net executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(Unknown Source)
	at java.net.SocketInputStream.read(Unknown Source)
	at java.io.BufferedInputStream.fill(Unknown Source)
	at java.io.BufferedInputStream.read(Unknown Source)
	at java.io.DataInputStream.readInt(Unknown Source)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:774)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
